In [1]:
%matplotlib inline

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.stats as sp
from os import walk
from collections import Counter

#4
import networkx.algorithms.community as nxcom

#5
import powerlaw # https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0085777
import networkx.algorithms.distance_measures as nxdm

#6
from NEMtropy import UndirectedGraph, DirectedGraph
from NEMtropy.network_functions import build_adjacency_from_edgelist

#### Utilities

In [ ]:
def mean_dictionary(dicto) : 
    return np.mean(list(dicto.values()))

def values_toarray(dicto):
    return np.array(list(dicto.values()))

def all_files(path) :
    names = []
    for files in walk(path):
        names.append(files)
    return names[0][2]

#### 1

In [3]:
def load_graph(path):
    try :
        return nx.read_gml(path)
    except nx.NetworkXError :
        return nx.read_graphml(path)

def degree_vec(graph):
    return np.asarray([d for d in dict(graph.degree()).values()])

def avg_degree(graph) :
    return (2 * graph.number_of_edges()/graph.number_of_nodes())

def max_degree(graph):
    degree_vec = degrees(graph)
    maxvalue = np.max(degreevec)
    maxid = np.argmax(degreevec)
    return maxid, maxvalue

def density(graph) : 
    L=nx.adjacency_matrix(graph).toarray().sum()
    N = graph.number_of_nodes()
    return ((2*L)/(N*(N-1)))

#### 2

In [ ]:
def randomise_graph(graph) :
    return nx.algorithms.smallworld.random_reference(graph,niter = 1, connectivity=False)

def compute_assortativity_coeff(graph) : 
    '''
    Degree assortativity is the tendency for nodes of high degree (resp. low degree) in a graph 
    to be connected to high degree nodes (resp. to low degree ones).
    return float
    '''
    return nx.degree_assortativity_coefficient(graph)

def compute_clustering_coeff(graph) : 
    '''
    Measure of the degree to which nodes in a graph tend to cluster together.
    return dict
    '''
    return nx.clustering(graph)

def avg_clustering(graph) :
    '''
    return float
    '''
    return nx.average_clustering(graph)

#### 3

In [ ]:
def centrality(g) : 
    eigen_cen = nx.eigenvector_centrality(g)
    eigen_cen = values_toarray(eigen_cen)
    
    close_cen = nx.closeness_centrality(g)
    close_cen = values_toarray(close_cen)
    
    between_cen = nx.betweenness_centrality(g)
    between_cen = values_toarray(between_cen)
    
    degree_cen = nx.degree_centrality(g)
    degree_cen = values_toarray(degree_cen)
    
    return eigen_cen, close_cen, between_cen, degree_cen


def pearson_coeff(x,y) :
    return sp.pearsonr(x,y)[0]


def kendall_coeff(x,y) :
    return sp.kendalltau(x,y)[0]


def spearman_coeff(x,y) :
    return sp.spearmanr(x,y)[0]

def pagerank(g,alpha) : 
    return nx.pagerank(g, alpha)

#### 4

In [ ]:
def find_community_label_propagation(graph) :
    return [c for c in nxcom.label_propagation_communities(graph)]

def find_community_greedy_modularity(graph) :
    return  [c for c in nxcom.greedy_modularity_communities(graph)]

def create_community_colors(graph, comm) :
    colormap = plt.cm.Paired
    color = [colormap(i) for i in np.linspace(0.23,0.99,len(comm))]
    
    for v in graph.nodes():
        for i, c in zip(range(len(comm)), color):
            if v in comm[i]:
                graph.nodes[v]['color'] = c
                continue
    return graph

def watts_strogatz_graph(nodes, k, prob):
    return nx.watts_strogatz_graph(nodes, k, prob)

def average_shortest_path_length(graph):
    return nx.average_shortest_path_length(graph)

#### 5

In [ ]:
def exponential_dist(k,mu):
    prob=[]
    for i in range(len(k)):
        prob.append((mu**-1)*math.exp(-k[i]/mu))
    
    return k, prob


def poisson_dist(k,mu) :
    prob=[]
    unique=list(set(k))
    deg = sorted(unique)
    
    for i in range(len(deg)):
        prob.append(sp.poisson.pmf(deg[i],mu))
    
    return deg, prob


def poisson_dist2(k,mu) :
    prob=[]
    unique=list(set(k))
    deg = sorted(unique)
    
    for i in range(len(deg)):
        p1=(mu**k[i])
        p2=Decimal(math.factorial(k[i]))
        p3=math.exp(-mu)
        p=Decimal(p1*p3)/p2
        prob.append(p)
    
    return deg, prob

def diameter(graph):
    return nxdm.diameter(graph)

6

In [ ]:
def avg_reciprocity(graph) : 
    "Reciprocity for directed networks is the fraction of bidirectional links"
    return nx.reciprocity(graph)

def init_UndirectedGraph(g):
    adj=nx.to_numpy_array(g)
    return UndirectedGraph(adj)

def init_DirectedGraph(g):
    adj=nx.to_numpy_array(g)
    return DirectedGraph(adj)
    
def sample_networks(g, out_dir, model,samples, isDirected=True):
    
    '''
    
    Directed 
    models for binary network: "dcm" and "dcm_exp"
    models for weighted networks: "crema", "decm" and "decm_exp"
    
    Undirected
    models for binary networks: "cm" and "cm_exp"
    models for weighted networks: "crema", "ecm" and "ecm_exp"
    
    '''
    if isDirected :
        adjacency="dcm_exp"   
    else :
        adjacency="cm_exp"
        
    g.solve_tool(model=model, 
                 method="newton",
                 initial_guess="random",
                 adjacency=adjacency,
                 method_adjacency="newton")
    
    g.ensemble_sampler(samples, cpu_n=1, output_dir=out_dir)

#### Plotting

In [ ]:
def degree_distrib(graph, filename):
    fig, ax = plt.subplots()
    plt.title("deg distribution : " + filename)
    plt.ylabel("P(k) - Degree Distribution")
    plt.xlabel("k - Degree")
    
    degree_vec = degrees(graph)
    
    binlog = np.logspace(np.log10(min(degree)),np.log10(max(degree)+1), 30)
    x, bins, p=plt.hist(degree, binlog, density=True, alpha=0.5)
    plt.axvline(x=avg_degree(graph))
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    
def plot_avg_degree_NnVd(graph, filename) :
    x= nx.average_neighbor_degree(graph)
    y= degrees(graph)
    
    binlog = np.logspace(np.log10(min(y)),np.log10(max(y)+1), 30)
    x, bins, p=plt.hist(y, binlog, density=True, alpha=0.5)
    plt.title(filename)
    plt.xlabel("average degree nearest neighbor")
    plt.ylabel("vertices degrees")
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    
def plot_scatter(x, y,  xlabel, ylabel, title, x_scale="linear", y_scale="linear") :
    plt.scatter(x, y, s=10, c='r', marker="o")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xscale(x_scale)
    plt.yscale(y_scale)
    plt.show()
    
def plot_scatter_cent(x, y, xt, yt,filename) :
    vertices = np.arange(start=1, stop=len(x)+1, step=1) 
    plt.scatter(x, y, s=10, c='r', marker="o")
    plt.title(filename+" "+xt+", "+yt)
    
    idxX = np.where(x == np.amax(x))
    idxY = np.where(y == np.amax(y))
    
    plt.annotate(idxX, xy=(x[idxX], y[idxX]), xytext=(0,50), 
            textcoords='offset points', ha='center', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                            color='blue'))
    
    plt.annotate(idxY, xy=(x[idxY], y[idxY]), xytext=(0,50),
            textcoords='offset points', ha='center', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                            color='blue'))
    
    plt.xlabel(xt)
    plt.ylabel(yt)
    plt.xscale('linear')
    plt.yscale('linear')
    plt.show()
    print("Pearson : ", pearson_coeff(x, y))
    print("Kendall : ", kendall_coeff(x, y))
    print("Spearman : ", spearman_coeff(x, y))
    
def draw_graph_community_color(graph,filename,method) :
    pos = nx.spring_layout(graph)
    plt.figure(3, figsize=(10, 8))
    plt.title(filename + ", "+ method)
    nx.draw(graph,with_labels=False,node_color = [graph.nodes[v]['color'] for v in graph.nodes()], node_size=50, pos=pos)
    plt.show()


def plot(x, y, xlabel, ylabel, title, yline=None, xline=None, x_scale="linear", y_scale="linear", size=[20,20]) :
    plt.figure(3, figsize=(size[0], size[1]))
    plt.rcParams.update({'font.size': 14})
    plt.plot(x, y, c='r', marker="o")
    plt.title(title)
    if yline!=None :
        plt.axhline(y=yline)
    if xline!=None :
        plt.axvline(x=xline)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xscale(x_scale)
    plt.yscale(y_scale)
    plt.show()
    
    
def plot_cmm_x(x, data, xlabel, ylabel, labels, title, yline=None, xline=None, x_scale="linear", y_scale="linear", size=[10,10]) :
    plt.figure(3, figsize=(size[0], size[1]))
    plt.rcParams.update({'font.size': 14})
    
    color = iter(plt.cm.rainbow(np.linspace(0, 1, len(data))))

    for i in range(len(data)):
        plt.plot(x, data[i], c=next(color))
    
    plt.title(title)
    if yline!=None :
        plt.axhline(y=yline)
    if xline!=None :
        plt.axvline(x=xline)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xscale(x_scale)
    plt.yscale(y_scale)
    plt.legend(labels)
    plt.grid()
    plt.show()